https://www.packtpub.com/article-hub/fine-tuning-llama-2

In [1]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to c:\users\carth\appdata\local\temp\pip-req-build-kb0_ebn7
  Resolved https://github.com/huggingface/transformers to commit ece1b62b93cde70233f235f6a4c84e37bfc8eba0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\carth\AppData\Local\Temp\pip-req-build-kb0_ebn7'


In [2]:
pip install peft

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


Original imports

# Importing Libraries
from transformers import LlamaTokenizer, LlamaForCausalLM 
import torch
from datasets import 
Dataset import transformers 
import pandas as pd
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_int8_training, get_peft_model_state_dict, PeftModel 
from sklearn.utils import shuffle

In [4]:
import torch
import transformers 
import pandas as pd
from sklearn.utils import shuffle
from peft import get_peft_model, LoraConfig, TaskType, get_peft_model_state_dict, PeftModel, prepare_model_for_kbit_training
from transformers import LlamaTokenizer, AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset

In [5]:
df = pd.read_csv('SMSSpamCollection.csv', sep='\t', header=None, encoding='latin1')

In [6]:
df

,0
0,"v1,v2,,,"
1,"ham,""Go until jurong point, crazy.. Available ..."
2,"ham,Ok lar... Joking wif u oni...,,,"
3,"spam,Free entry in 2 a wkly comp to win FA Cup..."
4,"ham,U dun say so early hor... U c already then..."
...,...
5570,"spam,""This is the 2nd time we have tried 2 con..."
5571,"ham,Will Ì_ b going to esplanade fr home?,,,"
5572,"ham,""Pity, * was in mood for that. So...any ot..."
5573,"ham,The guy did some bitching but I acted like..."


In [7]:
all_text = df[0].str.lower().tolist()  
 
input, output = [], []  
for text in all_text:  
               input.append(text)  
               output.append({word: '10_WORDS_LONG' for word in text.split() if len(word)==10}) 
 
df = pd.DataFrame([input, output]).T 
df.rename({0:'input_text', 1: 'output_text'}, axis=1, inplace=True) 
print (df.head(5))

                                          input_text  \
0                                           v1,v2,,,   
1  ham,"go until jurong point, crazy.. available ...   
2               ham,ok lar... joking wif u oni...,,,   
3  spam,free entry in 2 a wkly comp to win fa cup...   
4  ham,u dun say so early hor... u c already then...   

                       output_text  
0                               {}  
1  {'wat...",,,': '10_WORDS_LONG'}  
2                               {}  
3  {'rate)t&c's': '10_WORDS_LONG'}  
4                               {}  


In [8]:
df.drop(0, axis=0, inplace=True)
df.reset_index(drop=True)

,input_text,output_text
0,"ham,""go until jurong point, crazy.. available ...","{'wat..."",,,': '10_WORDS_LONG'}"
1,"ham,ok lar... joking wif u oni...,,,",{}
2,"spam,free entry in 2 a wkly comp to win fa cup...",{'rate)t&c's': '10_WORDS_LONG'}
3,"ham,u dun say so early hor... u c already then...",{}
4,"ham,""nah i don't think he goes to usf, he live...","{'though"",,,': '10_WORDS_LONG'}"
...,...,...
5569,"spam,""this is the 2nd time we have tried 2 con...","{'spam,""this': '10_WORDS_LONG'}"
5570,"ham,will ì_ b going to esplanade fr home?,,,",{}
5571,"ham,""pity, * was in mood for that. so...any ot...","{'ham,""pity,': '10_WORDS_LONG'}"
5572,"ham,the guy did some bitching but i acted like...",{'interested': '10_WORDS_LONG'}


In [9]:
from huggingface_hub import login
login()

In [10]:
model_name = "meta-llama/Llama-2-7b-hf" 
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(model_name)
 

def create_peft_config(model): 
    peft_cofig = LoraConfig( 
    task_type=TaskType.CAUSAL_LM, 
    inference_mode=False, 
    r=8, 
    lora_alpha=16, 
    lora_dropout=0.05, 
    target_modules=['q_proj', 'v_proj'],) 
    model = prepare_model_for_kbit_training(model)
    model.enable_input_require_grads() 
    model = get_peft_model(model, peft_cofig) 
    model.print_trainable_parameters() 
    return model, peft_cofig
 
model, lora_config = create_peft_config(model) 
 
def generate_prompt(data_point): 
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 
    ### Instruction: 
    Extract entity from the given input: 
    ### Input: 
    {data_point["input_text"]} 
    ### Response: 
    {data_point["output_text"]}"""


tokenizer.pad_token_id = 0 
def tokenize(prompt, add_eos_token=True): 
    result = tokenizer( 
    prompt, 
    truncation=True, 
    max_length=128, 
    padding=False, 
    return_tensors=None,
    ) 
    if ( result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < 128 and add_eos_token):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1) 
        result["labels"] = result["input_ids"].copy() 
    return result


def generate_and_tokenize_prompt(data_point): 
    full_prompt = generate_prompt(data_point) 
    tokenized_full_prompt = tokenize(full_prompt) 
    return tokenized_full_prompt

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [11]:
total_ds = shuffle(df, random_state=42) 
total_train_ds = total_ds.head(4000) 
total_test_ds = total_ds.tail(1500)

train_examples = {"input_text": [], "output_text": []}
test_examples = {"input_text": [], "output_text": []}

for _, row in total_train_ds.iterrows():
    train_examples["input_text"].append(row["input_text"])
    train_examples["output_text"].append(row["output_text"])

for _, row in total_test_ds.iterrows():
    test_examples["input_text"].append(row["input_text"])
    test_examples["output_text"].append(row["output_text"])

total_train_ds_hf = Dataset.from_dict(train_examples)
total_test_ds_hf = Dataset.from_dict(test_examples)

tokenized_tr_ds = total_train_ds_hf.map(generate_and_tokenize_prompt) 
tokenized_te_ds = total_test_ds_hf.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [12]:
training_arguments = transformers.TrainingArguments(  
per_device_train_batch_size=1, gradient_accumulation_steps=16,  
learning_rate=4e-05,  
logging_steps=100,  
optim="adamw_torch",  
evaluation_strategy="steps",  
save_strategy="steps",  
eval_steps=100,  
save_steps=100,  
output_dir="saved_models/" 
) 
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer) 
trainer = transformers.Trainer(model=model, tokenizer=tokenizer, 
            train_dataset=tokenized_tr_ds, eval_dataset=tokenized_te_ds, args=training_arguments, 
               data_collator=data_collator) 
 
with torch.autocast("cuda"): 
       trainer.train()

C:\Users\carth\anaconda3\lib\site-packages\torch\amp\autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [14]:
from transformers import Trainer, TrainingArguments

# Define the TrainingArguments
training_arguments = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=4e-05,
    logging_steps=100,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=100,
    save_steps=100,
    output_dir="saved_models/"
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_arguments,
    train_dataset=tokenized_tr_ds,
    eval_dataset=tokenized_te_ds,
    data_collator=data_collator,
)

# Train the model
trainer.train()


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
# Print details of the first 5 examples in the tokenized training dataset
for i in range(min(5, len(tokenized_tr_ds))):  
    print(f"Example {i + 1}:")
    try:
        print("Input IDs:", tokenized_tr_ds[i]["input_ids"])
        print("Attention Mask:", tokenized_tr_ds[i]["attention_mask"])
        if "labels" in tokenized_tr_ds[i]:
            print("Labels:", tokenized_tr_ds[i]["labels"])
        else:
            print("Labels not available")
    except KeyError as e:
        print("Error:", e)  # Print the specific error if a key is missing
    print()

In [ ]:
print(tokenized_tr_ds[0])